In [1]:
from itertools import product
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
# from torch.nn.functional import relu
# from torch.nn.functional import tanh
# from torch.nn.functional import sigmoid
from scipy.fftpack import dct
from scipy.signal import convolve2d
from skimage.measure import block_reduce

## Reduce the data size by shrinking the image and allow symbolic dynamics to occur (based on doppler-effect).

### Load the data

In [2]:
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=True,
)
test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=ToTensor(),
)
data_len = {'train': len(train_data), 'test': len(test_data)}

loaders = {
    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),

    'test': DataLoader(test_data,
                       batch_size=100,
                       shuffle=True,
                       num_workers=1),
}

### Symbolic dynamics through iteration and non-linearity.
Through multiple iterations apply the doppler effect to the DCT of the images and apply convolution.  
Save the results in a dataframe for easy loading.

In [3]:
def relu(x):
    return np.maximum(0, x)
relu = np.vectorize(relu)
# sigmoid = np.vectorize(lambda x: 1 / (1 + np.exp(-x)))
tanh = np.vectorize(np.tanh)

def lrelu(x):
    l_ = 0.025
    return np.maximum(0, x) + l_ * np.minimum(0, x)
lrelu = np.vectorize(lrelu)

def rrelu(x):
    return np.maximum(0, x) + np.random.rand() / 10 * np.minimum(0, x)
rrelu = np.vectorize(rrelu)

def elu(x):
    l_ = 0.05
    return np.maximum(0, x) + np.minimum(l_ * np.expm1(x), 0)
elu = np.vectorize(elu)

In [4]:
def apply_dynamics(sample, iters, f_act, vel, v_o, v_m, kernel, conv=False, pool=False, p=1):
    working_sample = np.copy(sample)

    for i in range(iters):
        v_s = vel(i, iters)
        # apply doppler effect to sample
        working_sample = ((v_m + v_o) / (v_m + v_s)) * working_sample  # doppler-effect
        working_sample = f_act(working_sample)  # activation function

        if conv:
            working_sample = convolve2d(working_sample, kernel, mode='valid')  # convolution

        if pool and i % 4 == 1:
            working_sample = block_reduce(working_sample, (2, 2), cval=0.5, func=pool, func_kwargs={'p': p})

    return working_sample

###### Visualization
Changing `iters` determines the size of the output (if conv and/or pooling are on).

In [5]:
samples, labels = next(iter(loaders['train']))
sample = samples[0][0]

# vel_s is negative if moving towards observer
def v1(x, n):
    return -(x + 1) / (n / 3)
def v2(x, n):
    return x
def v3(x, n):
    return 1.5

v_o = 0  # positive if moving towards source
v_m = 5.022  # small tail to avoid division errors.
kernel = np.array([[0.25, 0.25],
                   [0.25, 0.25]])

def lppool(x, axis, p):
    return np.power(np.sum(np.power(x, p), axis) * 1 / 4, 1 / p)

def mixedpool(x, axis, p):
    return p * np.max(x, axis) + (1 - p) * np.mean(x, axis)

In [6]:
iters = 6
f_act = relu  # relu, tanh, lrelu, rrelu, elu
conv = True
pool = mixedpool  # False, lppool, mixedpool
p = 0.2

freq_sample = dct(dct(sample.numpy().T, norm='ortho').T, norm='ortho')  # decompose sample
print(f"freq_sample {freq_sample.shape}")#\n{np.round(freq_sample, 2)}")
result = apply_dynamics(freq_sample, iters, f_act, v2, v_o, v_m, kernel, conv=conv, pool=pool, p=p)
print(f"result {result.shape}\n{np.round(result, 6)}")

freq_sample (28, 28)
result (5, 5)
[[0.024091 0.016779 0.010254 0.004462 0.301437]
 [0.01286  0.009811 0.006545 0.003823 0.301553]
 [0.007141 0.005686 0.004318 0.003378 0.301488]
 [0.003889 0.003746 0.003433 0.002906 0.30119 ]
 [0.301655 0.301507 0.301296 0.300978 0.40044 ]]


###### Convert all data
Changing `iters` determines the size of the output.

In [7]:
df_results = pd.DataFrame(columns=['data', 'label', 'train'])
for dset in ["train", "test"]:
    batch_size = 100
    batches = data_len[dset] / batch_size
    print(f"Begin reducing {dset}")
    for b, (images, labels) in enumerate(loaders[dset]):
        if b == batches:
            break
        for i in range(batch_size):
            freq_sample = dct(dct(images[i].numpy()[0].T, norm='ortho').T, norm='ortho')  # decompose sample
            result = apply_dynamics(freq_sample, iters, f_act, v2, v_o, v_m, kernel, conv=conv, pool=pool, p=p)
            result_str = np.array2string(result, separator=',')
            df_results = pd.concat([df_results, pd.DataFrame([[result_str, labels[i].numpy(), dset == "train"]], columns=df_results.columns)], ignore_index=True)

        if (b + 1) % 30 == 0:
            print('Reduced [{}/{}] {} batches'
                  .format(b + 1, batches, dset))

print(df_results.shape)
df_results.to_csv(f'freq_RedData/freq_RedData_{iters}i_conv{"_" + pool.__name__ + str(p).replace(".", "") if pool else ""}_{f_act.__name__}.csv', index=False)

Begin reducing train
Reduced [30/600.0] train batches
Reduced [60/600.0] train batches
Reduced [90/600.0] train batches
Reduced [120/600.0] train batches
Reduced [150/600.0] train batches
Reduced [180/600.0] train batches
Reduced [210/600.0] train batches
Reduced [240/600.0] train batches
Reduced [270/600.0] train batches
Reduced [300/600.0] train batches
Reduced [330/600.0] train batches
Reduced [360/600.0] train batches
Reduced [390/600.0] train batches
Reduced [420/600.0] train batches
Reduced [450/600.0] train batches
Reduced [480/600.0] train batches
Reduced [510/600.0] train batches
Reduced [540/600.0] train batches
Reduced [570/600.0] train batches
Reduced [600/600.0] train batches
Begin reducing test
Reduced [30/100.0] test batches
Reduced [60/100.0] test batches
Reduced [90/100.0] test batches
(70000, 3)
